In [33]:
import sys
from datetime import datetime, timedelta
import numpy as np
import tqdm
import pandas as pd
import oandapy

sys.path.append('/Users/toshio/project/fx')
from config import token


class CandleFetcher(object):

    name = "USDJPY_M1"
    currency_pair = "USD_JPY"
    unit = "USD_JPY"
    granularity = "M1"
    oanda = oandapy.API(environment="practice", access_token=token)

    def fetch(self, start_t, end_t):
        save_name = "../intermediate_data/{}_{}_{}.pkl".format(
            self.name,
            start_t.strftime('%Y%m%d@%H%M%S'),
            end_t.strftime('%Y%m%d@%H%M%S'))
        dates = []
        minutes_cls = []
        minutes_max = []
        minutes_min = []
        minutes_opn = []
        nth = 0
        mod = (end_t - start_t)
        dmin = int(min(60 * 27 * 3 , max(mod.days * 24 * 60 + mod.seconds // 60, 1))) #M15以上のときは、60*27*3*15
        mod = timedelta(minutes=dmin)
        bar = tqdm.tqdm(range(int(np.ceil((end_t - start_t) / mod))))
        
        while start_t + mod * nth < end_t:
            start = datetime.strftime(start_t + mod * nth, '%Y-%m-%dT%H:%M:%SZ')
            end = datetime.strftime(min(start_t + mod * (nth + 1), end_t), '%Y-%m-%dT%H:%M:%SZ')
            response = self.oanda.get_history(
                instrument=self.currency_pair,
                granularity=self.granularity,
                start=start,
                end=end,
                candleFormat="midpoint",
            )

            for res in sorted(response["candles"], key=lambda x: x["time"]):
                md = datetime.strptime(res["time"][:-8], '%Y-%m-%dT%H:%M:%S')
                dates.append(md)
                minutes_opn.append(res["openMid"])
                minutes_cls.append(res["closeMid"])
                minutes_min.append(res["lowMid"])
                minutes_max.append(res["highMid"])

            data = {
                "date": dates,
                "open": minutes_opn,
                "close": minutes_cls,
                "high": minutes_max,
                "low": minutes_min,
            }
            bar.set_description("Get {} ~ {}".format(start, end))
            bar.update(1)
            nth += 1

        df = pd.DataFrame(data, columns=["date", "open", "close", "high", "low"])
        df.to_pickle(save_name)
        print(df.head(5))


if __name__ == "__main__":
    fet = CandleFetcher()
    fet.fetch(datetime(2010, 1, 1), datetime(2018, 8, 23))




























  0%|          | 0/936 [00:00<?, ?it/s]

4860
3 days, 9:00:00
935.1111111111111
2010-01-01T00:00:00Z 2010-01-04T09:00:00Z





























Get 2010-01-01T00:00:00Z ~ 2010-01-04T09:00:00Z:   0%|          | 0/936 [00:04<?, ?it/s]


























Get 2010-01-01T00:00:00Z ~ 2010-01-04T09:00:00Z:   0%|          | 1/936 [00:04<1:11:19,  4.58s/it]

2010-01-04T09:00:00Z 2010-01-07T18:00:00Z





























Get 2010-01-04T09:00:00Z ~ 2010-01-07T18:00:00Z:   0%|          | 1/936 [00:25<1:11:19,  4.58s/it]


























Get 2010-01-04T09:00:00Z ~ 2010-01-07T18:00:00Z:   0%|          | 2/936 [00:25<2:28:18,  9.53s/it]

...




























Get 2010-03-02T18:00:00Z ~ 2010-03-06T03:00:00Z:   2%|▏         | 18/936 [04:58<4:03:25, 15.91s/it]


























Get 2010-03-02T18:00:00Z ~ 2010-03-06T03:00:00Z:   2%|▏         | 19/936 [04:58<4:30:53, 17.72s/it]

2010-03-06T03:00:00Z 2010-03-09T12:00:00Z





























Get 2010-03-06T03:00:00Z ~ 2010-03-09T12:00:00Z:   2%|▏         | 19/936 [05:10<4:30:53, 17.72s/it]


























Get 2010-03-06T03:00:00Z ~ 2010-03-09T12:00:00Z:   2%|▏         | 20/936 [05:10<4:05:28, 16.08s/it]

2010-03-09T12:00:00Z 2010-03-12T21:00:00Z


KeyboardInterrupt: 